
### 1.1  Importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string

### 1.2 messages intuition
sms messages classification as spam or not spam using uci data set from our intuition about spam sms we usuallly know that they contain url about the offer or concerned website. SMS spam messages are usually bulk messages containing short a short description, url, an offer/deal which is targeting a large set of users so the messages are generally short;

In [ ]:
#importing sms data set
sms = pd.read_csv("../input/sms-spam-collection-dataset/spam.csv",encoding='latin-1')
sms.head()

sms = sms.rename(columns={"v1":"label", "v2":"message"})
sms.head()
sms.describe()
sms.groupby('label').describe()
sms = sms.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)

In [ ]:
sms.head()

In [ ]:
sms.describe()
sms.groupby('label').describe()

### 1.3 priliminary examination of the sms
we can infer that there are 747 spam messages and  4825 not spam messages and some messages have repeated more than once



### 1.4 Dropping duplicate sms in both spam and not spam

In [ ]:
sms.drop_duplicates(inplace=True)
sms.groupby('label').describe()

### 1.5 after dropping duplicate messages

the number of not spam messages dropped from 4825 to 4516 and and spam messages from 747 to 653

for further analysis from here we have more not spam messages than spam messages



### 1.6 To check if all messages have appropriate labels

It is necessary to check if all the 5169 messages have appropriate labels

In [ ]:
countspam=0
countham=0
unlabelled=0
for x in sms['label']:
    if x=='spam': 
        countspam=countspam+1
    elif x=='ham':
        countham=countham+1
    else:
        unlabeled=unlabeled+1
print("Total number of spam messages",countspam)        
print("Total number of ham messages",countham)  
print("Total number of unlabelled messages",unlabelled)  

### 1.7 Length of messages in both spam and not spam




In [ ]:
sms['length'] = sms['message'].apply(len)
sms.head();
plt.title('Histogram distribution of the spam and not spam messages')
sms['length'].hist(bins=60,figsize=(12,5))
plt.xlim(-40,900)
plt.xlabel('length')
plt.ylabel('frequency')
plt.show()



sms['length'].hist(by=sms['label'],bins=60,figsize=(12,5))
plt.xlim(-40,900);
plt.xlabel('length')
plt.ylabel('frequency')
plt.show()

#### 1.7.1 Inference from the length of sms
from the histogram distribution of both spam and not spam messages we can conclude that majority of the spam messages have a greater length. we can at this stage also conclude that if a message has a length greater than 150 we can infer that the sms will be a spam, but the accuracy of our ML model will be very less; hence for further analysis we should clean and preprocess the data;

It is important here to note that we calculated the total number of characters in the entire message that is if a message is "i'll call you back" thaen the total length of the message 18. But language is not character that includes spaces, letters but word as a whole, hence lets split the sentence snd find the exact number of words in a given message.

In [ ]:
# find number of words in the message
def word_counter(msg):
    no_words = sum([i.isalpha() for i in msg.split()])
    return no_words

In [ ]:
sms['word count']=sms['message'].apply(word_counter)


plt.title('Histogram distribution of number of words in both spam and not spam messages')
sms['length'].hist(bins=100,figsize=(12,5))
plt.xlabel('No. of words')
plt.ylabel('frequency')
plt.show()



sms['word count'].hist(by=sms['label'],bins=50,figsize=(12,5))
plt.xlabel('No. of words')
plt.ylabel('frequency')
plt.show()

#### 1.7.2 Inference from the length of sms


1. From the above graph we can clearly make out that ham(not spam) messages are primarily having less number of words from the graph word count between 0-20 have a higher frequency and usually simple short messages;
2. But spam messages have a lot of words beacause from the graph we can see that a lot of messages have a word count between 12-22 and have higher frequency in this interval.

### 1.8.1 Binarising labels has ham(not spam) as 0 and spam as 1

representing spam and not Spam(ham) as 1 and 0 makes it efficient for classifing messages

In [ ]:
sms['labels'] = sms['label'].map({'ham': 0, 'spam': 1})
sms.head()

# 2.0 SMS Cleaning and Wrangling

by simply going through the sms we see that they contain a lot of puntations, words which hinder the learning capabilities of the ML;
hence it is necessary to clean the sms;
to clean the sms:
1. remove punctuation using python's string 
2. remove word like 'him', 'and' which do not add much meaning to the message using stop words from nltk
3. re


#### 2.1 Delete punctuations by using python string

In [ ]:
# function to delete punctuations by python string 
print('List of supported punctuations',string.punctuation)
def delete_punctuation(msg):
    new_msg=''.join([p for p in msg if p not in string.punctuation])
    return new_msg
sms['del_p_message']=sms['message'].apply(delete_punctuation)

sms['del_p_length'] = sms['del_p_message'].apply(len)
sms.head()

### 2.1.1 Visulizing sms in terms of length before and after removing punctuations

In [ ]:
plt.title('Histogram distribution of total characters spam and not spam messages after deleting punctuations')
sms['del_p_length'].hist(bins=250,figsize=(12,5))
plt.xlim(-40,900)
plt.xlabel('length')
plt.ylabel('frequency')
plt.show()


sms['del_p_length'].hist(by=sms['label'],bins=200,figsize=(12,5))
plt.xlim(-40,900);
plt.xlabel('length')
plt.ylabel('frequency')
plt.show()




In [ ]:
sms['words after removing punct.']=sms['del_p_message'].apply(word_counter)


plt.title('Histogram distribution of number of words in both spam and not spam messages after removing punctuation')
sms['words after removing punct.'].hist(bins=100,figsize=(12,5))
plt.xlabel('No. of words')
plt.ylabel('frequency')
plt.show()



sms['words after removing punct.'].hist(by=sms['label'],bins=50,figsize=(12,5))
plt.xlabel('No. of words')
plt.ylabel('frequency')
plt.show()



### 2.2 Cleaning SMS by using STOP WORDS which are not informative enough to classify as spam or not spam

In [ ]:
from nltk.corpus import stopwords
stopwords.words("english")

# definie function to delete stop words
def delete_stopwords(msgp):
    msg_no_stopwords=' '.join([x for x in msgp.split() if x.lower() not in stopwords.words("english")])
    
    return msg_no_stopwords

sms['del_SW_message']=sms['del_p_message'].apply(delete_stopwords)



In [ ]:
sms['del_SW_length']=sms['del_SW_message'].apply(len)
sms['del_SW_message'].head()
sms.head()

## 2.2.1 SMS cleaning and wrangling Visulization and comparing histogram with Punctuation removed words



In [ ]:
plt.title('Histogram distribution of the spam and not spam messages after deleting Stop words')
sms['del_SW_length'].hist(bins=50,figsize=(12,5))
plt.xlim(-40,900)
plt.xlabel('length')
plt.ylabel('frequency')
plt.show()


sms['del_SW_length'].hist(by=sms['label'],bins=60,figsize=(12,5))
plt.xlim(-40,400);
plt.xlabel('length')
plt.ylabel('frequency')
plt.show()

sms['del_SW_length'].hist(bins=150,alpha=0.8,figsize=(12,5),label='no stopwords')
sms['del_p_length'].hist(bins=150,alpha=0.5,figsize=(12,5),label='punctuation length')
sms['length'].hist(bins=150,alpha=0.2,figsize=(12,5),label='true length')
plt.legend(loc='upper right')
plt.show()


In [ ]:
sms['words after removing SW']=sms['del_SW_message'].apply(word_counter)


plt.title('Histogram distribution of number of words in both spam and not spam messages after removing STOPWORDS')
sms['words after removing SW'].hist(bins=100,figsize=(12,5))
plt.xlabel('No. of words')
plt.ylabel('frequency')
plt.show()



sms['words after removing SW'].hist(by=sms['label'],bins=50,figsize=(12,5))
plt.xlabel('No. of words')
plt.ylabel('frequency')
plt.show()



#### 2.3 SMS word visulization using wordcount

In [ ]:
#pictorially represent spam and not spam words using WordCloud
from wordcloud import WordCloud
spam_words = ' '.join(list(sms[sms['labels'] == 1]['message']))
spam_wc = WordCloud(width = 512,height = 512).generate(spam_words)
plt.figure(figsize = (10, 8), facecolor = 'k')
plt.imshow(spam_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

notspam_words = ' '.join(list(sms[sms['labels'] == 0]['message']))
notspam_wc = WordCloud(width = 512,height = 512).generate(notspam_words)
plt.figure(figsize = (10, 8), facecolor = 'k')
plt.imshow(notspam_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()


#### 2.5 Stemming Words
Stemming is the process of giving a similar meaning to the words in a given sentence;
word like programming, programmed, programmer, program can be collectively called program

from nltk.stem import SnowballStemmer 
stemmer= SnowballStemmer("english")
def stemming(msg):
    stemmed_sentence =' '.join([x for x in msg in stemmer.stem(x)])
    return stemmed_sentence

In [ ]:
from nltk.stem import SnowballStemmer 
stemmer= SnowballStemmer("english")
def stemming(text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words =words + (stemmer.stem(i))+" "
    return words

In [ ]:
sms['stemmed_message']=sms['del_SW_message'].apply(stemming)
sms['stem_message_length']= sms['stemmed_message'].apply(len)
sms['stemmed_word_length']= sms['stemmed_message'].apply(word_counter)

In [ ]:
sms.head()

#### 2.5.1 Visulization of message length after removing stem words with histogram 

In [ ]:
plt.title('Histogram distribution of the spam and not spam messages after Stemming')
sms['stem_message_length'].hist(bins=150,figsize=(12,5))
plt.xlim(-40,700)
plt.xlabel('length')
plt.ylabel('frequency')
plt.show()


sms['stem_message_length'].hist(by=sms['label'],bins=150,figsize=(12,5))
plt.xlim(-40,700);
plt.xlabel('length')
plt.ylabel('frequency')
plt.show()


sms['length'].hist(bins=150,alpha=0.4,figsize=(12,5),label='true length msg')
sms['del_SW_length'].hist(bins=150,alpha=0.5,figsize=(12,5),label='no stopwords msg')
sms['stem_message_length'].hist(bins=150,alpha=0.6,figsize=(12,5),label='stem msg length')
plt.legend(loc='upper right')
plt.show()

#### 2.5.2 Visulization of Number of words after removing stem words with histogram 

In [ ]:
plt.title('Histogram distribution of number of words in both spam and not spam messages after stemming')
sms['stemmed_word_length'].hist(bins=100,figsize=(12,5))
plt.xlabel('No. of words')
plt.ylabel('frequency')
plt.show()



sms['stemmed_word_length'].hist(by=sms['label'],bins=50,figsize=(12,5))
plt.xlabel('No. of words')
plt.ylabel('frequency')
plt.show()


#### 2.5.3 Inference from Visulization of Number of words after removing stem words with histogram 
From the above graph we see that the number of words before and after doing preprocessing and cleaning the number of words in spam are much higher in the interval 10 - 20 and in the ham(not spam) the number of words in the interval 0-20 are higher
hence we can conclude that word count is a feature which can be further used in our ML model.

### 2.6 Tokenizing and Vectorizing words
Tokenizing is a process of giving a unique id to each word our data set;
it is easier to implement to implement machine learning models on numerical values which map a particular word
after Tokenizing the occurence of each word in our data set is calculted
certain words repeat a number of times and do not add much value to our dataset hence they have to dealt with by normalizing and weighing them


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
count_v=count_vector.fit_transform(sms['stemmed_message'])


#### 2.6.1 Term frequency inverse document frequency - TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

sms_tfidf = TfidfTransformer().fit_transform(count_v)

### 3.0 Splitting the words into training set and test test



In [ ]:
#splitting data into training and test data
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest=train_test_split(sms_tfidf,sms['labels'], random_state=1)
print('Training messages size',xtrain.shape[0])
print('Testing messages size',xtest.shape[0])

### 4.0 Applying Different ML models and tabulating resultsand finding the best model

In [ ]:
#training data using naive bayes from sklearn

from sklearn.naive_bayes import MultinomialNB
naive_bayes=MultinomialNB()
naive_bayes.fit(xtrain,ytrain)

In [ ]:
#testing data and making predictions
prediction=naive_bayes.predict(xtest)

#testing data and making predictions
prediction=naive_bayes.predict(xtest)


In [ ]:
#calculate metrics of the algorithm
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
print('accuracy score ',accuracy_score(ytest,prediction))


#### 4.1.1 Inference from naive bayes MultinomialNB() model
we have a receceived a high accuracy score of 95.82 %. now scale the test and train data with MinMax scaler if it further increases the accuracy of Naive Bayes ML model

In [ ]:
from sklearn.preprocessing import MinMaxScaler
xxtrain=xtrain.A
xxtest=xtest.A
scaler = MinMaxScaler()
scaled_xtrain = scaler.fit_transform(xxtrain)
scaled_xtest  = scaler.transform(xxtest)

In [ ]:
naive_bayes_with_scalling = MultinomialNB().fit(scaled_xtrain, ytrain)
pred_NB_scaled = naive_bayes_with_scalling.predict(scaled_xtest)
accuracy_NB_scalling = accuracy_score(ytest, pred_NB_scaled)
print('accuracy score with scalling ',accuracy_NB_scalling)

#### 4.1.2 Inference from naive bayes MultinomialNB() model after scalling
we have a receceived a high accuracy score of 98.37 % after scalling the messages with MinMaxScaler.
After visually studying the influence of number of words in spam and not spam messages let us fit with current model and see if it further increases the accuracy of the Naive Bayes ML model

In [ ]:
from scipy.sparse import  hstack
sms_stack_len = hstack((sms_tfidf ,np.array(sms['stemmed_word_length'])[:,None])).A

xl_train, xl_test, yl_train, yl_test = train_test_split(sms_stack_len,sms['labels'], random_state=1)

In [ ]:
naive_bayes_withlength=MultinomialNB()
naive_bayes_withlength.fit(xl_train,yl_train)
pred_naive_bayes_withlength=naive_bayes_withlength.predict(xl_test)
print('accuracy score with word count feature without scalling ',accuracy_score(yl_test,pred_naive_bayes_withlength))

In [ ]:
X2_tfidf_train = xl_train
X2_tfidf_test  =  xl_test

scaler = MinMaxScaler()
X2_tfidf_train = scaler.fit_transform(X2_tfidf_train)
X2_tfidf_test  = scaler.transform(X2_tfidf_test)

naive_bayes_with_sc_len = MultinomialNB().fit(X2_tfidf_train, yl_train)
pred_NB_sc_len = naive_bayes_with_sc_len.predict(X2_tfidf_test)
accuracy_NB_sc_len = accuracy_score(yl_test, pred_NB_sc_len)
print('accuracy score with word count feature with scalling',accuracy_NB_sc_len)

#### 4.1.3 Inference from naive bayes MultinomialNB() model after scalling and implementing Number of words
we have a receceived an accuracy score of 98.29 % after implementing number of words with the model. we can infer that the number of words almost had negligible effect on the ML model. without including the no of words feature we had a accuracy score of 98.37 %. hence for better results it better to use ML model without considering number of words feature